In [3]:
%matplotlib inline
from pycocotools.coco import COCO
import numpy as np
import skimage.io as io
import matplotlib.pyplot as plt
import pylab
import json
pylab.rcParams['figure.figsize'] = (8.0, 10.0)


In [2]:
dataDir='/Users/azakaria/Downloads'
dataType='val2017'
annFile=f'{dataDir}/annotations/instances_{dataType}.json'

In [4]:
# initialize COCO api for instance annotations
coco=COCO(annFile)

loading annotations into memory...
Done (t=0.72s)
creating index...
index created!


In [ ]:
# display COCO categories and supercategories
cats = coco.loadCats(coco.getCatIds())
cat_map = {}
for i,cat in enumerate(cats):
    cat_map[cat['id']] = i
#print(len(cats))
#print(coco.anns.keys()) #keys are the image ids I believe
#print(coco.anns[1768].keys())
nms=[cat['name'] for cat in cats]
#print(nms)
#print([cat['id'] for cat in cats])

In [6]:
catIds = coco.getCatIds()
def get_imgs_from_each_category(catIds, numImgs):
    imgIds = []
    for catId in catIds:
        imgIds_current_batch = coco.getImgIds(catIds=[catId])[:]
        if(len(imgIds_current_batch) < numImgs):
            print(f'category {catId}: {len(imgIds_current_batch)} total images. WARNING! Less than {numImgs} images threshold')
            #print(f'WARNING! Less than {numImgs} images in category {catId}! {len(imgIds_current_batch)} total images')
        else:
            print(f'category {catId}: {len(imgIds_current_batch)} total images')
        imgIds = imgIds + imgIds_current_batch
    return imgIds

In [7]:
from pathlib import Path
def coco_to_yolo_bb(x1, y1, w, h, image_w, image_h):
    #"bbox": [x,y,width,height], (COCO)
    #x_center y_center width height (YOLO)
    return [((2*x1 + w)/(2*image_w)) , ((2*y1 + h)/(2*image_h)), w/image_w, h/image_h]
ann_c = set()
def write_label_file(imgId):
    # using with statement
    img = coco.loadImgs(imgId)[0]
    label_dir = Path('/Users/azakaria/Code/openmpf-yolo-training/coco/dataset/labels')
    filename = img['file_name'].split('.')[0] + '.txt'
    #print(img['file_name'])
    with open(label_dir/filename, 'w') as file:
        for ann in coco.loadAnns(coco.getAnnIds(imgIds = imgId, catIds=catIds)):
            ann_c.add(cat_map[ann["category_id"]])
            if cat_map[ann["category_id"]] > 80:
                print(f'cat_id > 80 {ann["category_id"]}')
            x_center, y_center, width, height = coco_to_yolo_bb(*ann['bbox'], img['width'], img['height'])
            file.write(f'{cat_map[ann["category_id"]]} {x_center} {y_center} {width} {height}\n')
    return

In [8]:
import os
os.system('rm -rf /Users/azakaria/Code/openmpf-yolo-training/coco/dataset/images/*; rm -rf /Users/azakaria/Code/openmpf-yolo-training/coco/dataset/labels/*')
img_ids = get_imgs_from_each_category(catIds, 120)
for img_id in img_ids:
    write_label_file(img_id)

category 1: 2693 total images
category 2: 149 total images
category 3: 535 total images
category 4: 159 total images
category 5: 97 total images. WARNING! Less than 120 images threshold
category 6: 189 total images
category 7: 157 total images
category 8: 250 total images
category 9: 121 total images
category 10: 191 total images
category 11: 86 total images. WARNING! Less than 120 images threshold
category 13: 69 total images. WARNING! Less than 120 images threshold
category 14: 37 total images. WARNING! Less than 120 images threshold
category 15: 235 total images
category 16: 125 total images
category 17: 184 total images
category 18: 177 total images
category 19: 128 total images
category 20: 65 total images. WARNING! Less than 120 images threshold
category 21: 87 total images. WARNING! Less than 120 images threshold
category 22: 89 total images. WARNING! Less than 120 images threshold
category 23: 49 total images. WARNING! Less than 120 images threshold
category 24: 85 total images

In [ ]:
import os
import shutil

def copy_images_for_dataset_dir():
    src_image_dir = Path('/Users/azakaria/Downloads/val2017')
    fs = os.listdir('/Users/azakaria/Code/openmpf-yolo-training/coco/dataset/labels')
    for f in fs:
        if f == '.DS_Store':
            continue
        src = str(src_image_dir/f.split('.')[0]) + '.jpg'
        dest = '/Users/azakaria/Code/openmpf-yolo-training/coco/dataset/images/' + f.split('.')[0] + '.jpg'

        #print(str(src_image_dir/f.split('.')[0]) + '.jpg')
        shutil.copyfile(src, dest)
        #print(image_dir/f.split('.')[0])
    return
copy_images_for_dataset_dir()
os.system('/usr/local/bin/docker exec -ti 5dcf6e04dbba rm -rf /usr/src/datasets/coco_120_max_each_cat/images')
os.system('/usr/local/bin/docker exec -ti 5dcf6e04dbba rm -rf /usr/src/datasets/coco_120_max_each_cat/labels')
os.system('/usr/local/bin/docker cp /Users/azakaria/Code/openmpf-yolo-training/coco/dataset/. 5dcf6e04dbba:/usr/src/datasets/coco_120_max_each_cat')
os.system('ls /Users/azakaria/Code/openmpf-yolo-training/coco/dataset/images | wc -l ; ls /Users/azakaria/Code/openmpf-yolo-training/coco/dataset/labels | wc -l')